In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore')

# 📘 Dataset Overview

With thousands of universities worldwide, choosing the right one can be overwhelming. Students often rely on global rankings to guide their decisions—but rankings alone don’t tell the full story. Factors like research strength, teaching quality, international diversity, and industry partnerships all play a role in shaping an institution’s true value.  
This dataset, compiled by *Times Higher Education*, offers a detailed look into these metrics—making it a powerful tool for exploring what really defines the world’s top universities.


In [2]:
df = pd.read_csv('World University Rankings.csv')

In [3]:
df.head(10)

In [4]:
df.tail()

In [5]:
df.describe()

In [6]:
df.describe(include='O')

In [7]:
df.info()

In [8]:
df.isna().sum()

In [9]:
df.duplicated().sum()

In [10]:
df.nunique()

In [11]:
df[df['Year']==2025].sort_values(by='Rank', ascending=True)

In [12]:
df['Name'].value_counts()

# Cleaning Data

## International Students Missing Values

In [13]:
df['International Students'].value_counts()

In [14]:
x = list(set(df[df['International Students']=='%']['Name']))
x

In [15]:
df[df['Name'].isin(x)]

In [16]:
# Get valid (non-% values) for each university
valid_intl = df[df['International Students'] != '%'].groupby('Name')['International Students'].first()

df['International Students'] = df.apply(
    lambda row: valid_intl[row['Name']] if row['International Students'] == '%' and row['Name'] in valid_intl else row['International Students'],
    axis=1
)


In [17]:
df['International Students'] = df['International Students'].str.replace('%','').str.replace(' ','').astype(int) 


## Female to Male Ratio Nulls and Format 

In [18]:
df[df['Female to Male Ratio'].isnull()]['Name'].value_counts()

In [19]:
df['Female to Male Ratio'].value_counts()

In [20]:
df[df['Female to Male Ratio']=='1.011805556']['Name'].value_counts()

In [21]:
df[df['Name']=='Missouri University of Science and Technology']

In [22]:
df['Female'] = df['Female to Male Ratio'].apply(
    lambda x: x.split(':')[0].strip() if isinstance(x, str) and ':' in x else x
).astype('float')

df['Female'] = df['Female'].apply(lambda x: x*100 if x <= 1 else x)

df['Female'].value_counts()

In [23]:
mask = df['Female'] < 2

# Calculate the mean for each University, excluding the unwanted value
grouped_mean = df[~mask].groupby('Name')['Female'].mean()
df.loc[mask, 'Female'] = df.loc[mask, 'Name'].map(grouped_mean)

In [24]:
df['Female'].describe()

In [25]:
df[df['Female'].isnull()]['Name'].value_counts()

### Filling nulls by average of university then country

In [26]:
female_by_name = df.groupby('Name')['Female'].mean()
female_by_country = df.groupby('Country')['Female'].mean()

mask = df['Female'].isna()

df.loc[mask, 'Female'] = df.loc[mask, 'Name'].map(female_by_name)

# Step 4: Update mask (some values still NaN)
mask = df['Female'].isna()

df.loc[mask, 'Female'] = df.loc[mask, 'Country'].map(female_by_country)
df['Female'].describe()

In [27]:
df['Female'].value_counts()

In [28]:
df.isnull().sum()

### Splitting ratio into two columns

In [29]:
df['Female Ratio'] = df['Female'].round(0).astype(int)
df['Male Ratio'] = 100 - df['Female Ratio']

In [30]:
df['Female Ratio'].value_counts()

In [31]:
df['Male Ratio'].value_counts()

In [32]:
df = df.drop(columns=['Female to Male Ratio','Female'],axis=1)

In [33]:
df.describe()


## Detecting Outliers

In [34]:
outlier_columns = ['Student Population','Students to Staff Ratio']

for column in outlier_columns:
    sns.boxplot(x=df[column])
    plt.show()

In [35]:
top_population = df.sort_values(by='Student Population', ascending=False).head(15)
top_population

### Storing Extreme outliers for Analysis & Dropping from DF

In [36]:
big_universities = ['Anadolu University','Tribhuvan University','Payame Noor University']
special_population = df[df['Name'].isin(big_universities)]
special_population

In [37]:
df = df[~df['Name'].isin(big_universities)]
df.describe()

### Scaling for normalization

In [38]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

scaled = scaler.fit_transform(df[outlier_columns])

scaled_shifted = scaled - scaled.min(axis=0)

df[outlier_columns] = scaled_shifted

df.describe()

In [39]:
df['Year'].value_counts()

# Scraping iso_alpha code for countries

In [40]:
df_iso = pd.read_csv('countries_iso.csv')

df_iso.drop(columns=['Countries_iso_alpha_url'], inplace=True)
df_iso.rename(columns={'Countries_name': 'Country', 'Countries_iso_alpha': 'iso_alpha'}, inplace=True)
df_iso.head(10)


### Fixing Errors

In [41]:
corrections = {
    'United States of America (the)': 'United States',
    'United Kingdom of Great Britain and Northern Ireland (the)': 'United Kingdom',
    'Netherlands (Kingdom of the)': 'Netherlands',
    'Korea (the Republic of)': 'South Korea',
    'Russian Federation (the)': 'Russian Federation',
    'Taiwan (Province of China)': 'Taiwan',
    'Türkiye': 'Turkey',
    'Czechia': 'Czech Republic',
    'Iran (Islamic Republic of)': 'Iran',
    'United Arab Emirates (the)': 'United Arab Emirates',
    'Philippines (the)': 'Philippines',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Tanzania, the United Republic of': 'Tanzania',
    'Viet Nam': 'Vietnam',
    'Palestine, State of': 'Palestine',
    'Syrian Arab Republic (the)': 'Syria'
}


# Apply the corrections
df_iso['Country'] = df_iso['Country'].replace(corrections)

In [42]:
df_iso = df_iso.drop_duplicates(subset='Country')


df_universities = df.merge(
    df_iso[['Country','iso_alpha']],
    on='Country',
    how='left'
)


df_universities.describe(include='O')


In [43]:
df_universities.isna().sum()

In [44]:
df_universities[df_universities['iso_alpha'].isnull()]['Country'].unique()

### Filling Nulls

In [45]:
df_universities.loc[df_universities['Country'] == 'Northern Cyprus','iso_alpha'] = 'CYP'
df_universities.loc[df_universities['Country'] == 'Kosovo','iso_alpha'] = 'XKX'
df_universities.loc[df_universities['Country'] == 'Democratic Republic of the Congo','iso_alpha'] = 'DRC'



In [46]:
df_universities.isna().sum()

# Exploratory Data Analysis

In [47]:
df = df_universities
df.describe(include='O')

In [48]:
df.describe()

In [49]:
numeric_cols = list(df.select_dtypes(include='number').columns)
numeric_cols = [col for col in numeric_cols if col != 'Overall Score' and col != 'Year']
numeric_cols.append('Overall Score')
numeric_cols


In [50]:
numeric_df = df[numeric_cols]
numeric_df['Rank']= -numeric_df['Rank'] # for sensible relations
numeric_df.head()

## Correlation Matrix

In [51]:
# Compute the correlation matrix
corr_matrix = numeric_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 8))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)

plt.title("Correlation Heatmap of Numeric Features")
plt.tight_layout()
plt.show()


## Score vs Rank

In [52]:
plt.figure(figsize=(12, 8))
plt.scatter(numeric_df['Overall Score'],numeric_df['Rank'])
plt.title("Relation between Overall Score and Rank")
plt.show()

## International Diversity

In [53]:
plt.figure(figsize=(12, 8))
sns.scatterplot(df,x="International Outlook",y="International Students")

## Top Universities For Each Country

In [54]:
df[df['Rank']<11]['Name'].value_counts()

In [55]:
top_universities = list(df[df['Rank']<11]['Name'].unique())
top_df = df[df['Name'].isin(top_universities)]
top_df['Rank'] = - top_df['Rank']

fig3 = px.scatter(top_df,
                 x="Overall Score", y="Rank",
                 animation_frame="Year",
                 animation_group="Name",
                 size="Male Ratio",
                 color="Name",
                 hover_name="Name",
                 title="Top Universities Scores over Years")
fig3.show()

In [56]:
df.columns

In [57]:
pivot_gender_ratio = df[df['Year'].isin([2024,2025])].pivot_table(
    values='Female Ratio',
    index='Country',
    columns='Year',
    aggfunc='mean'
)
pivot_gender_ratio

In [58]:
# Sort by Country and Overall Score (descending)
df_sorted = df.sort_values(['Country', 'Overall Score'], ascending=[True, False])

# Group by Country and take top 100 for each
top100_per_country = df_sorted.groupby('Country').head(100).reset_index(drop=True)

top100_per_country =top100_per_country.sort_values(by='Overall Score', ascending=False)
top100_per_country.describe(include='O')

In [59]:
pivot_research = top100_per_country.pivot_table(
    values=['Research Quality', 'Research Environment'],
    index='Country',
    aggfunc='mean'
).sort_values(by='Research Quality', ascending=False)
pivot_research

In [60]:
pivot_international = top100_per_country.pivot_table(
    values=['International Outlook', 'International Students'],
    index='Country',
    aggfunc='mean'
).sort_values(by='International Students', ascending=False)
pivot_international

In [61]:
pivot_score_country = top100_per_country.pivot_table(
    values='Overall Score',
    index='Country',
    aggfunc='mean'
).sort_values(by='Overall Score', ascending=False)
pivot_score_country

In [62]:
fig = px.choropleth(top100_per_country, locations="iso_alpha",
                    color="Overall Score",
                    hover_name="Country",
                    title="Overall Score for each country")

# Show the map
fig.show()

In [63]:
top100_per_country = top100_per_country.sort_values('Year')

fig2 = px.scatter(top100_per_country,
                 x="Overall Score", y="International Students",
                 animation_frame="Year",
                 animation_group="Country",
                 color="Female Ratio",
                 hover_name="Country",
                 title="Countries Average Scores over Years")
fig2.show()

# Transformation to Excel File

In [64]:
df_main = pd.read_csv('World University Rankings.csv')
df_main = df_main[~df_main['Name'].isin(big_universities)]

df['Student Population'] = df_main['Student Population']
df['Students to Staff Ratio'] = df_main['Students to Staff Ratio']
# 
# with pd.ExcelWriter("world_universities_ranking.xlsx") as writer:
#     df.to_excel(writer, sheet_name='Cleaned Data', index=False)
#     pivot_score_country.to_excel(writer, sheet_name='Score per Country')
#     pivot_gender_ratio.to_excel(writer, sheet_name='Female Ratio by Year')
#     pivot_research.to_excel(writer, sheet_name='Research Environment & Quality')
#     pivot_international.to_excel(writer, sheet_name='International Outlook & Students')
#     special_population.to_excel(writer, sheet_name='Highest Population Universities')


In [65]:
df_sorted = df.sort_values(['Country', 'Overall Score'], ascending=[True, False])

# Group by Country and take top 100 for each
top100_per_country = df_sorted.groupby('Country').head(100).reset_index(drop=True)

top100_per_country =top100_per_country.sort_values(by='Overall Score', ascending=False)


top_per_year = df.groupby('Year').head(300).reset_index(drop=True)


with pd.ExcelWriter("Top Universities per country.xlsx") as writer:
    top100_per_country.to_excel(writer,sheet_name='top per country', index=False)
    top_per_year.to_excel(writer,sheet_name='top per year', index=False)